In [11]:
import torch
import sys
print(sys.path)
import pandas as pd
import numpy as np 
print(torch.__version__)

import numpy as np
import pandas as pd
from tsai.all import *

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

['/media/wen/vingt/NUS_MODULES/sph6004/assignment2/for_submission', '', '/opt/ros/melodic/lib/python2.7/dist-packages', '/home/wen/bus_ws/src/spconv/spconv', '/media/wen/vingt/det_methods/Continental_Radar/second.pytorch/second', '/media/wen/vingt/det_methods/Continental_Radar/second.pytorch', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '/home/wen/virtualenv_py/sph/lib/python3.8/site-packages']
1.11.0+cu102


In [12]:
## Get static and dynamic training & testing dataset 

input_date = '0407'
input_impute = 'knn_impute' #'with_missing' #mean_impute
train_static_path = 'data/train set/train_static_{}.csv'.format(input_date)
df_train_static = pd.read_csv(train_static_path)

train_dyn_path = 'data/train set/train_dynamic_{}_{}.csv'.format(input_impute, input_date)
df_train_dynamic = pd.read_csv(train_dyn_path)


test_static_path = 'data/test set/test_static_{}.csv'.format(input_date)
df_test_static = pd.read_csv(test_static_path)

test_dyn_path = 'data/test set/test_dynamic_{}_{}.csv'.format(input_impute, input_date)
df_test_dynamic = pd.read_csv(test_dyn_path)


# df_train_static = df_train_static[df_train_static['vent_start_hour']<35]
# df_test_static = df_test_static[df_test_static['vent_start_hour']<35]

print('Train set sizes:', df_train_static.shape, df_train_dynamic.shape)
print('Test set sizes:',df_test_static.shape, df_test_dynamic.shape)

assert len(df_train_static[df_train_static['vent_start_hour']>60]) ==0 
assert len(df_test_static[df_test_static['vent_start_hour']>60]) ==0 

Train set sizes: (1212, 22) (4220, 23)
Test set sizes: (515, 22) (1791, 23)


In [13]:
print("There are {} patients with vent start hour greater than 60".format(len(df_test_static[df_test_static['vent_start_hour']>60])))

There are 0 patients with vent start hour greater than 60


In [ ]:
ax = df_train_static.plot.hist(column=["vent_start_hour_1", "vent_end_hour_1"], bins=12, figsize=(10, 8),  range=[0, 60], alpha=0.5)
ax.set_title('Ventilation timing')
ax.set_xlabel('Hours')


In [39]:

def get_sequences(df_dyn_data, df_static, len_series):
    '''
    function takes in dynamic and static data and len series. 
    1. For each patient we parse the data for each patient with the chart time 
    2. Generate a subsequence for each patient of length : len_series
    '''
    count_ = np.zeros((60,1))
    labels = np.array([]).reshape(0,2)
    dyna_series_train =[]
    count = 0 
    num_df_sample = 0 

    for i, x in enumerate(df_static['stay_id']):
    #     print(x)
        df_sample = df_dyn_data[df_dyn_data['stay_id']==x] 

        num_df_sample+=(len(df_sample))
        df_label = df_static[df_static['stay_id']==x]    
        temp_sample = df_label.loc[:,['vent_start_hour_1', 'vent_duration_1']].values
        labels = np.vstack((labels, temp_sample)) #.values[0]  
        #for idx_s in range(len(df_sample)):

        if len(df_sample) < len_series: 
            while len(df_sample)< len_series: 
                df_dyn_data.loc[-1, 'stay_id'] = x
                df_dyn_data = df_dyn_data.sort_values('stay_id').reset_index(drop=True)
                df_dyn_data = df_dyn_data.interpolate()
                df_sample = df_dyn_data[df_dyn_data['stay_id']==x] 
            df_sample = df_sample.interpolate()

        zeroed_sample = df_sample[-len_series:].iloc[:, 3:].values
        assert zeroed_sample.shape[0] ==len_series
        dyna_series_train.append(zeroed_sample)

    temp = torch.tensor(dyna_series_train)
    train_dyn_set = temp.permute(0,2,1)
    # labels[:,0] = (labels[:,0] - 36)/60
    # print(count)
    #labels_tensor = torch.from_numpy(np.round(labels))
    return train_dyn_set, labels



def normalise_dyn(df_dyn):
    normalise_idx = 6
    df_dyn = df_dyn.drop(['charttime_hour_2', 'charttime_hour_1' , 'charttime_hour_6' , 'charttime_hour_12' , 'charttime_hour_24'], axis=1)
    df_dyn.iloc[:, 3:]  = (df_train_dynamic.iloc[:, 3:] - df_train_dynamic.iloc[:, 3:].mean()) / df_train_dynamic.iloc[:, 3:].std()

    return df_dyn



In [40]:


class MIMICDataset(Dataset):
    '''
    function generates a torch data loader type for training
    '''
    def __init__(self, data_dyn , data_labels):
        self.data = data_dyn
        self.labels_all = data_labels
 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        data_sampled = self.data[idx, :]
        label_sampled =  self.labels_all[idx,:]       
        sample = {'data': data_sampled , 'labels':label_sampled }
        return sample



In [16]:


class Net(nn.Module):
    def __init__(self, len_series, a= 20, d =10):
        super().__init__()
        #self.conv1 = nn.Conv1d(15, 20, 2)
        self.conv1 = nn.Conv1d(15, a, 1)
        self.bn1 = nn.BatchNorm1d(a)
        self.conv2 = nn.Conv1d(a, d, 1)
        self.pool = nn.MaxPool2d(1, 1)
        self.fc1 = nn.Linear(len_series*d, 4)
        # self.fc1 = nn.Linear(120, 4) #len series 5
        # self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        # x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [36]:

def evaluate_models(str_input , pred):
    eval_results = []
    ## str_input should be : start, end, duration 
    test_static_path = 'data/test set/test_static_{}.csv'.format(input_date)
    df_test_static = pd.read_csv(test_static_path)
    for num_hours in [1, 2, 6, 12, 24]:
        vent_hour_ =  'vent_{}_{}'.format(str_input, num_hours)  
        if num_hours ==1: 
            labels =  df_test_static[vent_hour_]
            L1_abs_err  = np.mean(abs(labels - pred))
            print('Mean L1 error:', np.round(L1_abs_err,2))
            eval_results.append(np.round(L1_abs_err,2))
            MSE  = np.mean((labels -pred )**2)
            print('Mean Square error:', np.round(MSE,2))
            eval_results.append( np.round(MSE,2))
        output_hours  = np.round(pred/num_hours)
        acc_hours = np.sum(output_hours == df_test_static[vent_hour_])/ len(df_test_static)*100
        eval_results.append(np.round(acc_hours,2))
        print('Accuracy at {} hours : {} %'.format(num_hours, np.round(acc_hours,2)))
    return eval_results


def train_model (model_name, c_in, lr, drop_out, len_series, isPCA, param_name ):

    print('Model training: param - ', param_name)
    loss_count = 0 
    loss_init = 0 
    loss_graph = []; ve_1 = []; ve_2 = []
    d = 200 
    if model_name =='CNN':
        print('Using CNN')
        net = Net(len_series)
    elif model_name == 'InceptionTime': 
        # print('Using Inception Time')
        # net = InceptionTime(c_in=c_in,c_out=2, seq_len=len_series) #, nf=a, depth = d)
        print('Using Inception Time with dropouts and batch norm')
        net =  InceptionTimePlus (c_in=c_in, c_out=2, seq_len=len_series, fc_dropout=0.05,
                    bn=False, ks=40, bottleneck=True, separable=False, dilation=1, stride=1,
                    conv_dropout=drop_out, sa=False, se=None, norm='Batch')
    elif model_name == 'TST':
        net = TST(c_in=15, c_out=2, seq_len=len_series, d_model=128, n_heads=16,  dropout=drop_out, n_layers=3,
            fc_dropout=0.1)


    optimizer = optim.Adam(net.parameters(), lr=lr)
    net.to(device)

    criterion = nn.MSELoss()
    # criterion = nn.L1Loss()
    # criterion = nn.SmoothL1Loss()
    # optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
   
    # optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    # scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.001)
    for epoch in range(200):  # loop over the dataset multiple times
        net.train()
        running_loss = 0.0
        for data_all in train_loader:
            inputs = data_all['data'].to(device)
            gt = data_all['labels'].float().to(device)
            optimizer.zero_grad()
            outputs = net(inputs.float())
            # loss = criterion(outputs , gt) 
            loss = criterion(outputs[:,0] , gt[:,0] ) +   criterion(outputs[:,1], gt[:,1])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        if abs(loss_init- running_loss/len(train_set) ) < 0.01:
            loss_count +=1
        if loss_count > 5:
            break
        loss_init = running_loss/len(train_set)
        loss_graph.append(running_loss/len(train_set))
        
        net.eval() 
        outputs = net(dyn_test.float().to(device))
        label_all = torch.from_numpy(np.array(labels_test)).float().to(device)

        val_error = torch.mean(abs(torch.round(outputs[:,0]) - label_all[:,0]))
        ve_1.append(val_error.cpu().detach().numpy().tolist())
        val_error2 = torch.mean(abs(torch.round(outputs[:,0]) - label_all[:,0]))
        ve_2.append(val_error2.cpu().detach().numpy().tolist())
        if epoch % 20 == 0:
            # print(f'[{epoch + 1}] loss: {running_loss/len(train_set):.3f}')
            val_error = torch.mean(abs(torch.round(outputs[:,0]) - label_all[:,0]))
            # print('Mean L1 error:', val_error)


    outputs = net(dyn_test.float().to(device))
    outputs = outputs.cpu().detach().numpy()
    temp = []
    print('===================  VENTILATION START TIME  ============================= ')
    eval_results = evaluate_models('start_hour', outputs[:,0] )
    if param_name == 'lr':  
        temp.extend([lr])
    elif param_name == 'drop_out':
        temp.extend([drop_out])
    elif param_name == 'len_series':
        temp.extend([len_series])
    
    temp.extend(eval_results)
    print('===================  VENTILATION DURATION TIME  ============================= ')
    eval_results2 = evaluate_models('duration', outputs[:,1] )
    temp.extend(eval_results2)
    total = round(eval_results[6] + eval_results2[6], 2)
    if (total) > 50:
        print('Saving model..')
        torch.save(net.state_dict(), 'best_model_inception_time_{}_withPCA_{}.pt'.format(total,isPCA))
    else: 
        # torch.save(net, 'curr_model_inception_time_{}_withPCA_{}.pt'.format(eval_results2[3]))
        print('No model saved - current model, performance : {}'.format(total) )
    print('Finished Training')
    return temp, outputs, loss_graph, ve_1, ve_2

## Grid search function

In [38]:

info = []
lr = 0.001
len_series = 4
model_name = 'InceptionTime'
drop_out = 0.05 
isPCA = False 

loss_graph_arr =[]
ve_1_arr= [] 
ve_2_arr =[]

param_name = 'drop_out'
params = {'lr': [0.1, 0.01, 0.001],  'len_series' :  [1, 2, 3, 4, 5, 10 , 15],  'drop_out' : [0, 0.05, 0.1, 0.15, 0.2], 'pca':[0, 1]}
for lr  in params[param_name]:
    print('Training:', param_name)

    df_train_dynamic_norm = normalise_dyn(df_train_dynamic)
    df_test_dynamic_norm = normalise_dyn(df_test_dynamic)

    dyn_train, labels_train = get_sequences(df_train_dynamic_norm, df_train_static, len_series)
    print(dyn_train.shape, labels_train.shape)
    dyn_test, labels_test = get_sequences(df_test_dynamic_norm, df_test_static, len_series)
    print(dyn_test.shape, labels_test.shape)
    c_in = 15
    
    batch_size =100
    train_set = MIMICDataset(dyn_train, labels_train)
    test_set = MIMICDataset(dyn_test, labels_test)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=1)
    train_size = len(train_set)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=1)
    test_size = len(test_set)
    data_loaders = {"train": train_loader, "test": test_loader}
    dataset_sizes = {"train": train_size, "test": test_size}
    print(dyn_train.shape, labels_train.shape)
    print(dyn_test.shape, labels_test.shape)
    print('C IN', c_in)

    ## Train the model, get the outputs, loss graph and errors for start of ventilation (ve_1) and duration (ve_2)
    temp, outputs, loss_graph, ve_1, ve_2 = train_model (model_name= model_name, c_in = c_in, lr=lr, drop_out = drop_out, len_series = len_series , isPCA= isPCA, param_name = param_name)
    loss_graph_arr.append(loss_graph)
    ve_1_arr.append(ve_1) 
    ve_2_arr.append(ve_2)
    info.append(temp)
    df_tuning = pd.DataFrame(info, columns=['drop_out', 'L1', 'MSE', 'start_1', 'start_2', 'start_6',  'start_12', 'start_24', 'vent_L1' , 'vent_MSE' , 'vent_1'  , 'vent_2'  , 'vent_6'  , 'vent_12'  , 'vent_24' ])
    df_tuning.to_csv('tuning_0414_it_{}_nopca.csv'.format(param_name))  
    # return outputs, loss_graph, ve_1, ve_2 

print('Finished searching..')

Training: drop_out
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
C IN 15
Training: drop_out
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
C IN 15
Training: drop_out
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
C IN 15
Training: drop_out
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
C IN 15
Training: drop_out
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
torch.Size([1212, 15, 4]) (1212, 2)
torch.Size([515, 15, 4]) (515, 2)
C IN 15


## Script for visualising loss functions

In [ ]:
fig,ax = plt.subplots(1,3,figsize = (15,5))
label =  params[param_name]
title_ = ['Loss', 'Start L1 Err', 'Duration L1 err']
y_label_ = ['Loss', 'Error', 'Error']
for idx in range(0, 3):
    num_epochs  = range(len(loss_graph))
    if idx == 0: 
        for enum_l, loss_graph in enumerate(loss_graph_arr): 
            ax[idx].plot(num_epochs, loss_graph, label = param_name + '= '+ str(label[enum_l])) 
    elif idx == 1: 
        for enum_l, ve_1 in enumerate(ve_1_arr):       
            ax[idx].plot(num_epochs, ve_1, label = param_name +'= '+ str(label[enum_l])) 
        ax[idx].set_xlim(0,150)
        ax[idx].set_ylim(0,150)
    elif idx == 2: 
        ax[idx].set_xlim(0,150)
        for enum_l, ve_2 in enumerate(ve_2_arr):  
            ax[idx].plot(num_epochs, ve_2, label = param_name+ ' = '+ str(label[enum_l]))
    ax[idx].set_xlabel('Epoch')
    ax[idx].set_ylabel('{}'.format(y_label_[idx]))

    ax[idx].title.set_text('{}'.format(title_[idx]))
    ax[idx].legend()

## Visualise the loss and test set error graphs

In [ ]:
def fig_plots(loss_graph, ve_1, ve_2):
    fig,ax = plt.subplots(1,3,figsize = (15,5))
    title_ = ['Loss', 'Start L1 Err', 'Duration L1 err']
    y_label_ = ['Loss', 'Error', 'Error']
    for idx in range(0, 3):
        num_epochs  = range(len(loss_graph))
        if idx == 0: 
            ax[idx].plot(num_epochs, loss_graph) 
        elif idx == 1: 
            ax[idx].plot(num_epochs, ve_1) 
        elif idx == 2: 
            ax[idx].plot(num_epochs, ve_2)
        ax[idx].set_xlabel('Epoch')
        ax[idx].set_ylabel('{}'.format(y_label_[idx]))

        ax[idx].title.set_text('{}'.format(title_[idx]))




## Evaluation Script 

In [ ]:

print('===================  VENTILATION START TIME  ============================= ')
evaluate_models('start_hour', outputs[:,0] )

print('===================  VENTILATION DURATION TIME  ============================= ')
evaluate_models('duration', outputs[:,1] )